In [9]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

馬名:horses_text
枠番:waku_num
馬番:nums
性別:sexes
年齢:ages
負担重量:weights
騎手:jockeys
タイム:times
馬体重:h_weights
馬体重増減:h_weight_diffs
調教師名:trainers
単勝人気:pops

距離:distance
コース(芝など):course
右or左回り:direction
開始時間(時):start_h
開始時間(分):start_m
天気:weather
日付(年):year
日付(月):month
日付(日):day

In [2]:
df = pd.DataFrame()

In [3]:
r = requests.get('https://jra.jp/datafile/seiseki/replay/2021/001.html')
soup = BeautifulSoup(r.content, 'html.parser')

horses = soup.find_all('td', class_='horse')

In [4]:
print(horses)

[<td class="horse">
        ヒシイグアス</td>, <td class="horse">
        ココロノトウダイ</td>, <td class="horse">
        ウインイクシード</td>, <td class="horse">
        ロザムール</td>, <td class="horse">
        アールスター</td>, <td class="horse">
        テリトーリアル</td>, <td class="horse">
        シークレットラン</td>, <td class="horse">
        ロードクエスト</td>, <td class="horse">
<span class="horse_icon"><img alt="マルチ" src="/JRADB/img/kigo/maru-chi.png"/></span>マウントゴールド</td>, <td class="horse">
        バイオスパーク</td>, <td class="horse">
        カデナ</td>, <td class="horse">
        ヴァンケドミンゴ</td>, <td class="horse">
        ショウナンバルディ</td>, <td class="horse">
        ディープボンド</td>, <td class="horse">
        リュヌルージュ</td>, <td class="horse">
<div class="horse">
	          	マイネルサーパス<div class="icon blinker"><img alt="ブリンカー着用" src="/JRADB/img/kigo/icon_blinker.png"/></div>
</div>
</td>, <td class="horse">
<span class="horse_icon"><img alt="マルガイ" src="/JRADB/img/kigo/maru-gai.png"/></span>ダーリントンホール</td>]


In [5]:
horses_text = [horse.text for horse in horses]

In [6]:
def name_modifier(name):
    name = name.strip()
    name = name.strip('\t')
    name = name.strip('\n')
    return name

In [7]:
horses_text = list(map(name_modifier, horses_text))

In [8]:
waku = soup.find_all('td', class_='waku')

In [9]:
waku_num = []
for w in waku:
    m = re.search(r'.+".(\d).".+', str(w))
    waku_num.append(m.group(1))

In [10]:
waku_num

['5',
 '2',
 '8',
 '6',
 '5',
 '1',
 '7',
 '1',
 '4',
 '8',
 '3',
 '7',
 '3',
 '6',
 '4',
 '2',
 '8']

In [11]:
ages_pre = soup.find_all('td', class_='age')

In [12]:
sexes, ages = [], []
for a in ages_pre:
    m = re.search(r'.+(牡|牝)(\d).+', str(a))
    # male or female (male : 0, female : 1, error : -1)
    if m.group(1) == '牡': sexes.append(0)
    elif m.group(1) == '牝': sexes.append(1)
    else: sexes.append(-1)
        
    # append age    
    ages.append(int(m.group(2)))

In [13]:
ages

[5, 4, 7, 5, 6, 7, 5, 8, 8, 6, 7, 5, 5, 4, 6, 5, 4]

In [14]:
sexes

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [15]:
weights_pre = soup.find_all('td', class_='weight')

In [16]:
weights = []
for w in weights_pre:
    m = re.search(r'.+t">(.+)</t.*', str(w))
    weights.append(float(m.group(1)))

In [17]:
jockeys_pre = soup.find_all('td', class_="jockey")

In [18]:
jockeys = []
for j in jockeys_pre:
    m = re.search(r'.+y">(.+)</t.*', str(j))
    jockeys.append(m.group(1))

In [19]:
times_pre = soup.find_all('td', class_='time')

In [20]:
times_pre

[<td class="time">2:00.9</td>,
 <td class="time">2:00.9</td>,
 <td class="time">2:01.2</td>,
 <td class="time">2:01.4</td>,
 <td class="time">2:01.4</td>,
 <td class="time">2:01.4</td>,
 <td class="time">2:01.6</td>,
 <td class="time">2:01.7</td>,
 <td class="time">2:01.7</td>,
 <td class="time">2:01.8</td>,
 <td class="time">2:01.8</td>,
 <td class="time">2:01.8</td>,
 <td class="time">2:01.9</td>,
 <td class="time">2:02.0</td>,
 <td class="time">2:02.1</td>,
 <td class="time">2:02.3</td>,
 <td class="time">2:03.6</td>]

In [21]:
times = []
for t in times_pre:
    m = re.search(r'.+e">(\d+):(.+)</t.+', str(t))
    time = float(m.group(1)) * 60 + float(m.group(2))
    times.append(time)

In [22]:
h_weights_pre = soup.find_all('td', class_='h_weight')

In [23]:
h_weights, h_weight_diffs = [], []
for hw in h_weights_pre:
    m = re.search(r'.+t">(\d+)<span>\((.+)\).+', str(hw))
    h_weights.append(int(m.group(1)))
    h_weight_diffs.append(int(m.group(2)))

In [24]:
trainers_pre = soup.find_all('td', class_= 'trainer')

In [25]:
trainers = []
for t in trainers_pre:
    m = re.search(r'.+r">(.+)</t.+', str(t))
    trainers.append(m.group(1))

In [26]:
pops_pre = soup.find_all('td', class_='pop')

In [27]:
pops = []
for p in pops_pre:
    m = re.search(r'.+p">(\d+)</t.+', str(p))
    pops.append(int(m.group(1)))

In [28]:
nums_pre = soup.find_all('td', class_='num')

In [29]:
nums = []
for n in nums_pre:
    m = re.search(r'.+m">(\d+)</t.+', str(n))
    nums.append(int(m.group(1)))

In [30]:
dist = soup.find_all('div', class_='cell course')

In [31]:
dist

[<div class="cell course">
 <span class="cap">コース：</span>2,000<span class="unit">メートル</span><span class="detail">（芝・右）</span></div>]

In [32]:
str(dist[0])

'<div class="cell course">\n<span class="cap">コース：</span>2,000<span class="unit">メートル</span><span class="detail">（芝・右）</span></div>'

In [33]:
m = re.search(r'.+コース：</span>(.+)<span class="u.+l">（(.)・(.).+', str(dist[0]))

In [34]:
distance = int(m.group(1)[:1] + m.group(1)[2:])

In [35]:
course = m.group(2)
direction = m.group(3)

In [36]:
start_time_pre = soup.find_all('div', class_='cell time')

In [37]:
start_time_pre

[<div class="cell time">
         発走時刻：<strong>15時35分</strong>
 </div>]

In [38]:
m = re.search(r'.+ng>(\d+)時(\d+)分.+', str(start_time_pre[0]))

In [39]:
start_h = int(m.group(1))

In [40]:
start_m = int(m.group(2))

In [41]:
weather_pre = soup.find_all('div', class_='cell baba')

In [42]:
weather_pre

[<div class="cell baba">
 <ul>
 <li class="weather"><span class="inner"><span class="cap">天候</span><span class="txt">曇</span></span></li><li class="turf"><span class="inner"><span class="cap">芝</span><span class="txt">良</span></span></li></ul>
 </div>]

In [43]:
m = re.search(r'.+天候</span><span class="txt">(.).+', str(weather_pre[0]))

In [44]:
weather = m.group(1)

In [45]:
date_pre = soup.find_all('div', class_='cell date')

In [46]:
date_pre

[<div class="cell date">2021年1月5日（火曜） 1回中山1日</div>]

In [47]:
m = re.search(r'e">(\d+)年(\d)月(\d)日.+', str(date_pre[0]))
year = int(m.group(1))
month = int(m.group(2))
day = int(m.group(3))

# DataFrame

In [48]:
import pandas as pd

In [49]:
distance_df = [distance for i in range(len(horses_text))]
course_df = [course for i in range(len(horses_text))]
direction_df = [direction for i in range(len(horses_text))]
start_h_df = [start_h for i in range(len(horses_text))]
start_m_df = [start_m for i in range(len(horses_text))]
weather_df = [weather for i in range(len(horses_text))]
year_df = [year for i in range(len(horses_text))]
month_df = [month for i in range(len(horses_text))]
day_df = [day for i in range(len(horses_text))]

In [50]:
df_list = [horses_text, waku_num, nums, sexes, ages, weights, jockeys, h_weights, h_weight_diffs,
          trainers, pops, distance_df, course_df, start_h_df, start_m_df, weather_df, 
           year_df, month_df, day_df, times]

In [51]:
df_list

[['ヒシイグアス',
  'ココロノトウダイ',
  'ウインイクシード',
  'ロザムール',
  'アールスター',
  'テリトーリアル',
  'シークレットラン',
  'ロードクエスト',
  'マウントゴールド',
  'バイオスパーク',
  'カデナ',
  'ヴァンケドミンゴ',
  'ショウナンバルディ',
  'ディープボンド',
  'リュヌルージュ',
  'マイネルサーパス',
  'ダーリントンホール'],
 ['5',
  '2',
  '8',
  '6',
  '5',
  '1',
  '7',
  '1',
  '4',
  '8',
  '3',
  '7',
  '3',
  '6',
  '4',
  '2',
  '8'],
 [9, 3, 16, 12, 10, 2, 13, 1, 7, 17, 6, 14, 5, 11, 8, 4, 15],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [5, 4, 7, 5, 6, 7, 5, 8, 8, 6, 7, 5, 5, 4, 6, 5, 4],
 [54.0,
  53.0,
  56.0,
  52.0,
  56.0,
  56.5,
  54.0,
  56.0,
  56.0,
  57.0,
  58.0,
  56.0,
  54.0,
  56.0,
  52.0,
  56.5,
  55.0],
 ['松山 弘平',
  '丸山 元気',
  '戸崎 圭太',
  '北村 宏司',
  '長岡 禎仁',
  '石川 裕紀人',
  '横山 武史',
  '西村 淳也',
  '内田 博幸',
  '池添 謙一',
  '三浦 皇成',
  '藤岡 康太',
  '津村 明秀',
  '和田 竜二',
  '中井 裕二',
  '丹内 祐次',
  'M.デムーロ'],
 [496,
  528,
  498,
  502,
  508,
  476,
  504,
  466,
  464,
  478,
  480,
  472,
  444,
  492,
  468,
  484,
  536],
 [2, 10, 0, 8, 4, 0, 4, 4, -2, 6, 2, 4,

In [52]:
df_r = pd.DataFrame(df_list).T

In [53]:
df_r.columns = ['horse', 'waku', 'umaban', 'sex', 'age', 'weight', 'jockey', 'horse_weight', 'horse_weight_diff',
             'trainer', 'pop', 'distance', 'course', 'start_h', 'start_m', 'weather', 'year', 'month', 'day', 'time'] 

In [59]:
df.columns = ['horse', 'waku', 'umaban', 'sex', 'age', 'weight', 'jockey', 'horse_weight', 'horse_weight_diff',
             'trainer', 'pop', 'distance', 'course', 'start_h', 'start_m', 'weather', 'year', 'month', 'day', 'time'] 

ValueError: Length mismatch: Expected axis has 0 elements, new values have 20 elements

In [56]:
df.head()

""


In [57]:
race_num = '1'.zfill(3)

In [58]:
race_num

'001'

In [60]:
len(df_list)

20

In [63]:
[[] for i in range(21)]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [64]:
a, b = [1, 2, 3], [4, 5, 6]

In [68]:
a = a + b

In [69]:
a

[1, 2, 3, 4, 5, 6, 4, 5, 6]

In [1]:
import scraping

In [2]:
df = scraping.get_dataset(21)

race 1
race 2
race 3
race 4
race 5
race 6
race 7
race 8
race 9
race 10
race 11
race 12
race 13
race 14
race 15
race 16
race 17
race 18
race 19
race 20
race 21


In [3]:
df

,horse,waku,umaban,sex,age,weight,jockey,horse_weight,horse_weight_diff,trainer,...,distance,course,direction,start_h,start_m,weather,year,month,day,time
0,ヒシイグアス,5,9,0,5,54,松山 弘平,496,2,堀 宣行,...,2000,芝,右,15,35,曇,2021,1,5,120.9
1,ココロノトウダイ,2,3,0,4,53,丸山 元気,528,10,手塚 貴久,...,2000,芝,右,15,35,曇,2021,1,5,120.9
2,ウインイクシード,8,16,0,7,56,戸崎 圭太,498,0,鈴木 伸尋,...,2000,芝,右,15,35,曇,2021,1,5,121.2
3,ロザムール,6,12,1,5,52,北村 宏司,502,8,上原 博之,...,2000,芝,右,15,35,曇,2021,1,5,121.4
4,アールスター,5,10,0,6,56,長岡 禎仁,508,4,杉山 晴紀,...,2000,芝,右,15,35,曇,2021,1,5,121.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,ブラックムーン,8,15,0,9,56,藤井 勘一郎,502,-4,西浦 勝一,...,1400,芝,右,15,35,晴,2021,2,28,80.2
317,メイショウキョウジ,8,16,0,6,56,酒井 学,484,2,藤岡 健一,...,1400,芝,右,15,35,晴,2021,2,28,80.3
318,カツジ,2,3,0,6,57,武 豊,504,4,池添 兼雄,...,1400,芝,右,15,35,晴,2021,2,28,80.3
319,ベストアクター,8,17,-1,7,56,団野 大成,456,8,鹿戸 雄一,...,1400,芝,右,15,35,晴,2021,2,28,80.4


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   horse              321 non-null    object
 1   waku               321 non-null    object
 2   umaban             321 non-null    object
 3   sex                321 non-null    object
 4   age                321 non-null    object
 5   weight             321 non-null    object
 6   jockey             321 non-null    object
 7   horse_weight       321 non-null    object
 8   horse_weight_diff  321 non-null    object
 9   trainer            321 non-null    object
 10  pop                321 non-null    object
 11  distance           321 non-null    object
 12  course             321 non-null    object
 13  direction          321 non-null    object
 14  start_h            321 non-null    object
 15  start_m            321 non-null    object
 16  weather            321 non-null    object
 1

In [7]:
df.iloc[0, 'weights']

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [19]:
r = requests.get('https://jra.jp/datafile/seiseki/replay/2021/jyusyo.html')
soup = BeautifulSoup(r.content, 'html.parser')
urls_pre = soup.find_all('td', class_='result')

In [13]:
urls_pre

[<td class="result"><a class="btn-def btn-xs" href="/datafile/seiseki/replay/2020/001.html"><i aria-hidden="true" class="fa fa-chevron-circle-right"></i>レース結果</a></td>,
 <td class="result"><a class="btn-def btn-xs" href="/datafile/seiseki/replay/2020/002.html"><i aria-hidden="true" class="fa fa-chevron-circle-right"></i>レース結果</a></td>,
 <td class="result"><a class="btn-def btn-xs" href="/datafile/seiseki/replay/2020/003.html"><i aria-hidden="true" class="fa fa-chevron-circle-right"></i>レース結果</a></td>,
 <td class="result"><a class="btn-def btn-xs" href="/datafile/seiseki/replay/2020/004.html"><i aria-hidden="true" class="fa fa-chevron-circle-right"></i>レース結果</a></td>,
 <td class="result"><a class="btn-def btn-xs" href="/datafile/seiseki/replay/2020/005.html"><i aria-hidden="true" class="fa fa-chevron-circle-right"></i>レース結果</a></td>,
 <td class="result"><a class="btn-def btn-xs" href="/datafile/seiseki/replay/2020/006.html"><i aria-hidden="true" class="fa fa-chevron-circle-right"></i>レー

In [20]:
urls = []
for url in urls_pre:
    m = re.search(r'.+href="(.+)"><i.+', str(url))
    if m is None: break
    urls.append(m.group(1))

In [21]:
urls

['/datafile/seiseki/replay/2021/001.html',
 '/datafile/seiseki/replay/2021/002.html',
 '/datafile/seiseki/replay/2021/003.html',
 '/datafile/seiseki/replay/2021/004.html',
 '/datafile/seiseki/replay/2021/005.html',
 '/datafile/seiseki/replay/2021/006.html',
 '/datafile/seiseki/replay/2021/007.html',
 '/datafile/seiseki/replay/2021/008.html',
 '/datafile/seiseki/replay/2021/009.html',
 '/datafile/seiseki/replay/2021/010.html',
 '/datafile/seiseki/replay/2021/011.html',
 '/datafile/seiseki/replay/2021/012.html',
 '/datafile/seiseki/replay/2021/013.html',
 '/datafile/seiseki/replay/2021/014.html',
 '/datafile/seiseki/replay/2021/015.html',
 '/datafile/seiseki/replay/2021/016.html',
 '/datafile/seiseki/replay/2021/017.html',
 '/datafile/seiseki/replay/2021/018.html',
 '/datafile/seiseki/g1/feb/result/feb2021.html',
 '/datafile/seiseki/replay/2021/019.html',
 '/datafile/seiseki/replay/2021/020.html',
 '/datafile/seiseki/replay/2021/021.html']